## 1. Problem Description

In this tutorial we will use a standard machine learning problem called the *iris flowers dataset*.

This dataset is well studied and is a good problem for practicing on neural networks because all of the $4$ input variables are numeric and have the same scale in centimeters. Each instance describes the properties of an observed flower measurements and the output variable is specific iris species.

This is a multi-class classification problem, meaning that there are more than two classes to be predicted, in fact there are three flower species. This is an important type of problem on which to practice with neural networks because the three class values require specialized handling.

The iris flower dataset is a well-studied problem and a such we can expect to achieve a model accuracy in the range of $95\%$ to $97\%$. This provides a good target to aim for when developing our models.

You can download the iris flowers dataset from the UCI Machine Learning repository and place it in your current working directory with the filename “iris.csv“.

## 2. Import Classes and Functions

We can begin by importing all of the classes and functions we will need in this tutorial.

This includes both the functionality we require from Keras, but also data loading from pandas as well as data preparation and model evaluation from scikit-learn.

In [1]:
#Tutorial from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


## 3. Initialize Random Number Generator

Next, we need to initialize the random number generator to a constant value (7).

This is important to ensure that the results we achieve from this model can be achieved again precisely. It ensures that the stochastic process of training a neural network model can be reproduced.

In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

## 4. Load The Dataset

The dataset can be loaded directly. Because the output variable contains strings, it is easiest to load the data using pandas. We can then split the attributes (columns) into input variables (X) and output variables (Y).

In [3]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

## 5. Encode The Output Variable

The output variable contains three different string values.

When modeling multi-class classification problems using neural networks, it is good practice to reshape the output attribute from a vector that contains values for each class value to be a matrix with a boolean for each class value and whether or not a given instance has that class value or not.

This is called *one hot encoding* or *creating dummy variables* from a categorical variable.

For example, in this problem three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the observations:


> Iris-setosa

> Iris-versicolor

> Iris-virginica

We can turn this into a one-hot encoded binary matrix for each data instance that would look as follows:


> Iris-setosa,	Iris-versicolor,	Iris-virginica

>$1\quad\quad\quad\quad\quad 0\quad\quad\quad\quad\quad 0$

> $0\quad\quad\quad\quad\quad 1\quad\quad\quad\quad\quad 0$

> $0\quad\quad\quad\quad\quad 0\quad\quad\quad\quad\quad 1$

We can do this by first encoding the strings consistently to integers using the scikit-learn class **LabelEncoder**. Then convert the vector of integers to a one hot encoding using the Keras function **to_categorical()**.

In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

**Note:** The variable encoded_Y has all outputs with labels $0,1,2$. When we use the categorical function, we get the variable dummy_y with labels $[1,0,0], [0,1,0], [0,0,1]$.

## 6. Define The Neural Network Model

The Keras library provides wrapper classes to allow you to use neural network models developed with Keras in scikit-learn.

There is a KerasClassifier class in Keras that can be used as an Estimator in scikit-learn, the base type of model in the library. The KerasClassifier takes the name of a function as an argument. This function must return the constructed neural network model, ready for training.

Below is a function that will create a baseline neural network for the iris classification problem. It creates a simple fully connected network with one hidden layer that contains $8$ neurons.

The hidden layer uses a rectifier activation function, which is a good practice. Because we used a one-hot encoding for our iris dataset, the output layer must create $3$ output values, one for each class. The output value with the largest value will be taken as the class predicted by the model.

Note that we use a “softmax” activation function in the output layer. This is to ensure the output values are in the range of $0$ and $1$ and may be used as predicted probabilities.

Finally, the network uses the efficient Adam gradient descent optimization algorithm with a logarithmic loss function, which is called “categorical_crossentropy” in Keras.

In [5]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

We can now create our KerasClassifier for use in scikit-learn.

We can also pass arguments in the construction of the KerasClassifier class that will be passed on to the **fit()** function internally used to train the neural network. Here, we pass the number of epochs as $200$ and batch size as $5$ to use when training the model. Debugging is also turned off when training by setting verbose to $0$.

In [6]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

## 7. Evaluate The Model with k-Fold Cross Validation

We can now evaluate the neural network model on our training data.

The scikit-learn has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation.

First we can define the model evaluation procedure. Here, we set the number of folds to be $10$ (an excellent default) and to shuffle the data before partitioning it.

**How k-fold cross-validation works:**
In k-fold cross-validation, the original sample (all the known data input-output) is randomly partitioned into $k$ equal sized subsamples. Of the $k$ subsamples, a single subsample is retained as the validation data for testing the model, and the remaining $k - 1$ subsamples are used as training data. The cross-validation process is then repeated $k$ times, with each of the $k$ subsamples used exactly once as the validation data. The $k$ results can then be averaged to produce a single estimation. The advantage of this method over repeated random sub-sampling (see below) is that all observations are used for both training and validation, and each observation is used for validation exactly once. $10$-fold cross-validation is commonly used, but in general $k$ remains an unfixed parameter.

In [7]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

Now we can evaluate our model (estimator) on our dataset (X and dummy_y) using a $10$-fold cross-validation procedure (kfold).

Evaluating the model only takes approximately $10$ seconds and returns an object that describes the evaluation of the $10$ constructed models for each of the splits of the dataset.

The results are summarized as both the mean and standard deviation of the model accuracy on the dataset. This is a reasonable estimation of the performance of the model on unseen data. It is also within the realm of known top results for this problem.

In [8]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 97.33% (4.42%)


Althought the result is great, note that this code only allow us to evaluate the accuracy of the model. We can't use it to make predictions. To make it possible we use the function **train_test_split** from scikit.learn. This function splits the data sample in two parts, and the parameter **test_size** sets the percentage of instances we reserve for testing (in this case we use $33\%$ for testing and $67\%$ for training). The rest will be used to train the model. Now the outputs predicted will be labeled as $0,1,2$, they are saved in the array **predictions**. Finally, we use the function **inverse_transform** to convert them to the actual labels of the flowers. 

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=seed)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)
print(predictions)
print(encoder.inverse_transform(predictions))

[2 1 0 1 2 0 1 1 0 1 1 1 0 2 0 2 2 2 0 0 1 2 1 2 2 2 1 1 2 2 2 1 0 2 1 0 0
 0 0 2 2 1 2 2 1 0 1 1 2 0]
['Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa']


In [10]:
#predictions entries are vectors the integers 0,1 or 2. We transform them in [1,0,0], [0,1,0] or [0,0,1].
predictions_dummy = np_utils.to_categorical(predictions)

#Compare predictions with the real outputs in Y_test to obtain the accuracy of the model in the test sample.
acc = sum(sum(predictions_dummy*Y_test))/len(Y_test)
print("Accuracy: ",acc*100,"%")

Accuracy:  98.0 %
